# Peyman hashemi
# 400131032

In [1]:
import numpy as np
import pandas as pd
import time
import scipy.io
import networkx as nx

In [4]:
dataset = pd.read_csv(r'C:\Users\Asus\Desktop\AUT\semister 3\complex\EX\ca-AstroPh.txt\ca-AstroPh.csv',na_values='?', comment='#', sep='\t', skipinitialspace=True, header=None)
N1 , N2 = dataset[0].tolist() , dataset[1].tolist()
list_edges = []
for i in range(len(N1)):
    list_edges.append((N1[i], N2[i]))
graph = nx.DiGraph(list_edges)
print(nx.info(graph))

Name: 
Type: DiGraph
Number of nodes: 18772
Number of edges: 396160
Average in degree:  21.1038
Average out degree:  21.1038


In [6]:
def graph_maker(graph, n_r, p):
    d_list = []
    
    for i in range(n_r):
        digraph = nx.DiGraph()
        digraph.add_nodes_from(list(graph.nodes))

        for edge in (list(graph.edges)):
            if np.random.uniform(0, 1, 1)[0] < p:
                digraph.add_edge(edge[0],edge[1])

        d_list.append(digraph)
    return d_list

In [7]:
list_r = graph_maker(graph , 10 , 0.01)
for l in list_r :
    print(nx.info(l))

Name: 
Type: DiGraph
Number of nodes: 18772
Number of edges: 4000
Average in degree:   0.2131
Average out degree:   0.2131
Name: 
Type: DiGraph
Number of nodes: 18772
Number of edges: 3896
Average in degree:   0.2075
Average out degree:   0.2075
Name: 
Type: DiGraph
Number of nodes: 18772
Number of edges: 3974
Average in degree:   0.2117
Average out degree:   0.2117
Name: 
Type: DiGraph
Number of nodes: 18772
Number of edges: 3993
Average in degree:   0.2127
Average out degree:   0.2127
Name: 
Type: DiGraph
Number of nodes: 18772
Number of edges: 4045
Average in degree:   0.2155
Average out degree:   0.2155
Name: 
Type: DiGraph
Number of nodes: 18772
Number of edges: 3913
Average in degree:   0.2084
Average out degree:   0.2084
Name: 
Type: DiGraph
Number of nodes: 18772
Number of edges: 4013
Average in degree:   0.2138
Average out degree:   0.2138
Name: 
Type: DiGraph
Number of nodes: 18772
Number of edges: 4010
Average in degree:   0.2136
Average out degree:   0.2136
Name: 
Type: DiG

In [14]:
def IC(graph_list, S):
    spread = []
    for i in range(len(graph_list)):
        current_ndoe, A = S[:], S[:]
        graph = graph_list[i]

        while current_ndoe:
            new_nodes = []
            for node in current_ndoe:
                new_nodes +=  list(set(graph.successors(node)))

            current_ndoe  = list(set(new_nodes) - set(A))
            A+= current_ndoe

        spread.append(len(A)-1)

    return (np.mean(spread))*0.25

In [9]:
def cost(S,graph_list):
    graph_lengh = len(graph_list)
    cost = 0
    for g in graph_list:
        for s in S:
            x = len(list(g.successors(s))) + len(list(g.predecessors(s)))
            if x == 0 :
                x+=1
            cost+= x
    Cost = (cost * 0.8) / graph_lengh       
    return Cost

In [33]:
# this function run "lazy hill climbing" Idea to speed up computing marginal gane
def lazy_hill_climbing(g, unit_benefit):
    #unit_benefit is unit cost or benefit ratio
    if unit_benefit:  # unit cost version :
        marginal_gain = [IC(g, [node]) for node in list(g[0].nodes)]
        
    else:  # benefit ration version :
        marginal_gain = [IC(g, [node]) / cost([node], g) for node in list(g[0].nodes)]
        
    Q = sorted(zip(list(g[0].nodes), marginal_gain), key=lambda x: x[1], reverse=True)
    
    S , s = [Q[0][0]] , Q[0][1]
    c = cost(S,g)
    
    if not unit_benefit :
        spread = c
    else : spread = 1
    counter_s = 1
    spread = spread * Q[0][1]
    print(str(Q[0][0]) + ": \nSpread = " + str(spread)+"\ncost = "+str(c))
    Q = Q[1:]
    flag = True
    while flag:
        check = False
        while not check:
            current = Q[0][0]
            if unit_benefit:  # unit cost version :
                Q[0] = (current, IC(g, S + [current])-s)
            else:  # benefit ratio version :
                Q[0] = (current, (IC(g, S + [current]) / cost(S + [current], g))-s)
            Q = sorted(Q, key=lambda x: x[1], reverse=True)
            check = (Q[0][0] == current)
        c = cost(S + [Q[0][0]], g)
        temp = Q[0][1]
        temp+=s
        if unit_benefit:
            temp /= c
        if 0.3 * (temp) > 1:
            spread = temp * c
            s+= Q[0][1]
            S.append(Q[0][0])
            counter_s += 1
            print("\nnumber " + str(counter_s) + ' : '+ str(Q[0][0]) + " : \nspread = "+str(spread)+" , \ncost = "+str(c))
            Q = Q[1:]
        else:
            nspread = temp * c
            print("\nnot add "+ str(Q[0][0]) + " : \nspread = "+str(nspread)+" \ncost = "+str(c))
            flag = False
    return S, spread

In [34]:
# this function run "cost-effective lazy-forward selection" algorithm
# unitCost_or_benefitRatio is a flag to change marginal gane to have theoretically guarantee
def CELF(g):
    start_time = time.time()
    print()
    print("             unit cost : \n")
    
    s_unit, spread_unit = lazy_hill_climbing(g, True)  # unit cost obj. func.
    time_unit_cost = [time.time() - start_time]
    print("time unit cost = ", time_unit_cost[0])
    print('result =',s_unit[0])
    start_time2 = time.time()
    print("========================================")
    print("             benefit Ratio :")
    
    s_benefit, spread_benefit = lazy_hill_climbing(g, False)  # benefit Ratio obj. func.
    time_benefit_ratio = [time.time() - start_time2]
    print("========================================")
    print('result of benefit Ratio = ',s_benefit)
    print("benefit-ratio finished = ")
    print("time benefit Ratio = ", time_benefit_ratio[0])
    if spread_benefit > spread_unit:
        S = s_benefit
        spread = spread_unit
    else:
        S = s_unit
        spread = spread_unit
    final_time = [time.time() - start_time]
    return S, spread, final_time

In [35]:
# Run CELF
save_IC_1 = []
S, spread, t = CELF(list_r)
print("=================================================")
print("             CELF Algorithm = ")
print("result of celf =  ",  S)
print("mean spread value = ",spread)
print("run time = ",t[0])


             unit cost : 

62821: 
Spread = 10.325
cost = 6.960000000000001

not add 53213 : 
spread = 18.775 
cost = 14.559999999999999
time unit cost =  1.029052972793579
result = 62821
             benefit Ratio :
118174: 
Spread = 7.425
cost = 0.9600000000000002

number 2 : 76391 : 
spread = 12.55 , 
cost = 1.7600000000000002

number 3 : 101558 : 
spread = 18.6 , 
cost = 2.72

number 4 : 133109 : 
spread = 23.05 , 
cost = 3.5200000000000005

number 5 : 85755 : 
spread = 25.7 , 
cost = 4.48

number 6 : 59103 : 
spread = 26.1 , 
cost = 5.28

number 7 : 46963 : 
spread = 27.449999999999996 , 
cost = 6.08

number 8 : 95952 : 
spread = 28.075 , 
cost = 7.279999999999999

number 9 : 78060 : 
spread = 29.025 , 
cost = 8.24

not add 84886 : 
spread = 30.125 
cost = 9.280000000000001
result of benefit Ratio =  [118174, 76391, 101558, 133109, 85755, 59103, 46963, 95952, 78060]
benefit-ratio finished = 
time benefit Ratio =  53.97298741340637
             CELF Algorithm = 
result of celf =  

In [41]:
def greedy_hill_climbing(g, k):
    S, spread, timelapse, start_time = [], [], [], time.time()
    exist = True
    while exist:
        best_spread = 0
        for j in set(g[0].nodes) - set(S):
            s = IC(g, S + [j])
            if s >= best_spread:
                best_spread, node = s, j
        Cost = cost(S + [node], g)
        if Cost<=k:
            timelapse.append(time.time() - start_time)
            S.append(node)
            print(str(node) +" , cost = "+str(Cost))  
        else:
            print("finished")
            exist = False
        spread.append(best_spread)
    return S, spread, timelapse

In [42]:
S, spread, duration = greedy_hill_climbing(list_r,10)
print("result of greedy algorithm =  ", S[0])

62821 , cost = 6.960000000000001
finished
result of greedy algorithm =   62821
